In [106]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import display

In [107]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
import math

In [108]:
df = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')

In [109]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [110]:
test.set_index('ID', inplace = True)
test.head()

,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [111]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [112]:
df.describe(exclude = None)

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [113]:
df.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

In [114]:
df.shape

(2935849, 6)

In [115]:
df['date'] = pd.to_datetime(df.date, format = '%d.%m.%Y')

In [116]:
df.sort_values('date')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
49800,2013-01-01,0,18,5823,2500.0,1.0
29784,2013-01-01,0,27,5573,849.0,1.0
35476,2013-01-01,0,7,1006,399.0,1.0
8330,2013-01-01,0,19,17707,899.0,1.0
57384,2013-01-01,0,14,19548,149.0,1.0
29791,2013-01-01,0,27,5574,699.0,2.0
54193,2013-01-01,0,8,16993,399.0,1.0
15899,2013-01-01,0,28,6468,449.0,1.0
10609,2013-01-01,0,19,13071,499.0,1.0
100980,2013-01-01,0,51,6450,483.0,1.0


In [117]:
test['date_block_num'] = 34
test['month'] = 11
test = test[['date_block_num', 'shop_id', 'item_id', 'month']]
test.head()

,date_block_num,shop_id,item_id,month
ID,,,,
0,34,5,5037,11
1,34,5,5320,11
2,34,5,5233,11
3,34,5,5232,11
4,34,5,5268,11


In [118]:
df1 = df.copy()
df1['month'] = df1['date'].dt.month
df1.drop(['date', 'item_price'], inplace = True, axis = 1)
df1 = df1[['date_block_num', 'shop_id', 'item_id', 'month', 'item_cnt_day']]
df1.head()

,date_block_num,shop_id,item_id,month,item_cnt_day
0,0,59,22154,1,1.0
1,0,25,2552,1,1.0
2,0,25,2552,1,-1.0
3,0,25,2554,1,1.0
4,0,25,2555,1,1.0


In [119]:
X = df1.drop('item_cnt_day', axis = 1)
y = df1['item_cnt_day']
display(X.head())
display(y.head())

,date_block_num,shop_id,item_id,month
0,0,59,22154,1
1,0,25,2552,1
2,0,25,2552,1
3,0,25,2554,1
4,0,25,2555,1


0    1.0
1    1.0
2   -1.0
3    1.0
4    1.0
Name: item_cnt_day, dtype: float64

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 1)

In [121]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [122]:
y_pred = reg.predict(X_test)

In [123]:
mean_squared_error(y_test,y_pred)

12.700773725556473

In [124]:
#Note: try to analyze seasonality, when sales were higher, etc.